# Bài tập lập trình: Thử nghiệm các mô hình phân loại cảm xúc văn bản
```
ProtonX - MLEs Class
```

# Cài đặt và import các thư viện cần thiết

In [ ]:
# !pip install transformers datasets
# !pip install accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 16.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.1/519.1 kB 25.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 268.8/268.8 kB 30.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 46.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 48.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 21.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 244.2/244.2 kB 1.3 MB/s eta 0:00:00


In [ ]:
# !pip install wandb

## **Quan trọng**: Nếu bạn sử dụng Colab, vui lòng khởi động lại thời gian chạy sau khi cài đặt xong hai thư viện này.

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer, DataCollatorWithPadding, TextClassificationPipeline, EarlyStoppingCallback
from datasets import load_dataset, load_from_disk, load_metric
import numpy as np

# Load model và tokenizer

Lựa chọn một trong 2 mô hình dưới đây:
- vinai/xphonebert-base
- FPTAI/vibert-base-cased

In [ ]:
model_name = "FPTAI/vibert-base-cased"

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained(model_name , num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name)

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at FPTAI/vibert-base-cased and are newly initialized: ['classifier.weight', 'classifier.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


# Dataset

Bộ dữ liệu phân loại cảm xúc bao gồm câu và nhãn tương ứng từ 1 đến 5

Ví dụ:

- Đầu vào: Mình thề là mình ko thể cảm nổi đồ ăn ở aeon mall Mình đã thử ăn từ nhà hàng đến food Ko có j ngon Sushi ăn tanh May mua có mấy miếng chứ mua hộp to chắc vứt đi Quảng cáo thì rõ rầm rộ xong khen lồng khen lộn lên Nhìn rõ đẹp chỉ phù hợp với sống ảo

- Nhãn: 4

In [ ]:
# Tải dataset
!gdown 1CriqmsH9MzKn9k78x9GvnH5UTAg3GBIF
!gdown 1RyAZ_vpO2sndK9ZTvJLenVMPcnIwJuhh

Downloading...
From: https://drive.google.com/uc?id=1CriqmsH9MzKn9k78x9GvnH5UTAg3GBIF
To: /content/negative_data.csv
100% 864k/864k [00:00<00:00, 57.8MB/s]
Downloading...
From: https://drive.google.com/uc?id=1RyAZ_vpO2sndK9ZTvJLenVMPcnIwJuhh
To: /content/positive_data.csv
100% 1.12M/1.12M [00:00<00:00, 73.8MB/s]


In [ ]:
dataset = load_dataset('csv', data_files={'negative_data.csv', 'positive_data.csv'})

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [ ]:
dataset.shuffle(seeds=20)

DatasetDict({
    train: Dataset({
        features: ['Rate', 'Review', 'Label'],
        num_rows: 4407
    })
})

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['Rate', 'Review', 'Label'],
        num_rows: 4407
    })
})


In [ ]:
dataset = dataset.remove_columns("Rate")

In [ ]:
dataset = dataset.rename_column("Review", "text")
dataset = dataset.rename_column("Label", "label")

In [ ]:
dataset = dataset['train']
def strip_text(example):
    example["text"] = example["text"].strip()
    return example

# Chạy hàm này trên tất cả các mẫu
dataset = dataset.map(strip_text)


Map:   0%|          | 0/4407 [00:00<?, ? examples/s]

In [ ]:
# Chia tập dữ liệu
dataset = dataset.train_test_split(test_size=0.1)

In [ ]:
print(dataset)

DatasetDict({
    train: Dataset({
        features: ['text', 'label'],
        num_rows: 3966
    })
    test: Dataset({
        features: ['text', 'label'],
        num_rows: 441
    })
})


In [ ]:
dataset['train'][0]

{'text': 'Tranquil là một quán cafe rất duyên và yên tĩnh Giá sách với nhiều thể không gian được bài trí tinh té và đồ uống chất',
 'label': 1}

In [ ]:
def tokenize_function(examples):
  return tokenizer(examples["text"], padding='max_length', truncation=True, max_length=256)

tokenized_datasets = dataset.map(tokenize_function, batched=True, remove_columns='text')

Map:   0%|          | 0/3966 [00:00<?, ? examples/s]

Map:   0%|          | 0/441 [00:00<?, ? examples/s]

In [ ]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 3966
    })
    test: Dataset({
        features: ['label', 'input_ids', 'token_type_ids', 'attention_mask'],
        num_rows: 441
    })
})

# Training

In [ ]:
metric = load_metric("accuracy")
def compute_metrics(eval_pred) :
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

<ipython-input-21-41be28b70558>:1: FutureWarning: load_metric is deprecated and will be removed in the next major version of datasets. Use 'evaluate.load' instead, from the new library 🤗 Evaluate: https://huggingface.co/docs/evaluate
  metric = load_metric("accuracy")


In [ ]:
training_args = TrainingArguments(
    output_dir="./results",
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=10,
    weight_decay=0.1,
    logging_strategy='epoch',
    evaluation_strategy='epoch',
    save_strategy='epoch',
    load_best_model_at_end=True,
)


trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
    callbacks = [EarlyStoppingCallback(early_stopping_patience=2)],
    # data_collator=data_collator,
)

trainer.train()

/usr/local/lib/python3.10/dist-packages/transformers/optimization.py:411: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(
You're using a BertTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


Epoch,Training Loss,Validation Loss,Accuracy
1,0.412900,0.283122,0.886621
2,0.235300,0.303813,0.913832
3,0.147000,0.381797,0.907029


TrainOutput(global_step=744, training_loss=0.26506370113741967, metrics={'train_runtime': 584.0453, 'train_samples_per_second': 67.906, 'train_steps_per_second': 4.246, 'total_flos': 1565247668336640.0, 'train_loss': 0.26506370113741967, 'epoch': 3.0})

In [ ]:
trainer.save_model("model/sequence_classification_huggingface/" + model_name)

In [ ]:
import os

from google.colab import drive
drive.mount('/content/drive')

!cp -r "model/sequence_classification_huggingface/" "/content/drive/MyDrive/model"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


**Kết quả mong đợi**: Với cả hai mô hình độ chính xác đều trên 91%

# Predict

In [ ]:
pipe = TextClassificationPipeline(model=model, tokenizer=tokenizer, return_all_scores=True, device=0)
pipe("Món ăn ở đây quá tệ")

/usr/local/lib/python3.10/dist-packages/transformers/pipelines/text_classification.py:104: UserWarning: `return_all_scores` is now deprecated,  if want a similar functionality use `top_k=None` instead of `return_all_scores=True` or `top_k=1` instead of `return_all_scores=False`.
  warnings.warn(


[[{'label': 'LABEL_0', 'score': 0.8565148711204529},
  {'label': 'LABEL_1', 'score': 0.14348511397838593}]]

So sánh kết quả training của hai mô hình này

|Mô hình   | Độ chính xác   |
|---|---|
|vinai/xphonebert-base   | 0.59637  |
| FPTAI/vibert-base-cased | 0.92063  |
|   |   |  


Chúc mừng bạn đã lập trình xong bài toán **phân loại mô hình cảm xúc.**